In [1]:
from tqdm.auto import tqdm
import numpy as np
import sample as sampler
import utils
import attacker
from model import BackBone

## Utils

In [2]:
def generate_puzzle_local_search_random(new_bs, new_isometric_matrixes, k = 160):
    """
    Behave as linear equation sampler.

    generate matrix with row vector from new_isometric_matrixes randomly,
    k indicates the number of row vectors to be selected from each isometric matrix, thus the total number of equations is k*len(new_isometric_matrixes)
    """
    sub_matrixes = []
    for b, isometric_matrix in zip(new_bs, new_isometric_matrixes):
        # get random subset of shape n of indexb
        indexbb = np.random.choice(b.shape[0], k, replace=False)
        sub_matrixes.append(isometric_matrix[indexbb, :])

    return np.concatenate(sub_matrixes, axis=0)

In [3]:
import time
def local_search_each_runtime(dimension, alpha, sub_matrix, disable_flag=False, each_runtime=1000):
    """
    Test local search run time of inner iteration. Return time = time of inner iteration * each_runtime(number of outer iteration).
    """
    times = 0
    # time start
    start = time.time()
    for _ in (pbar:=tqdm(range(each_runtime), disable=disable_flag, leave=True)):
        theta = sampler.sample_codeword(dimension, alpha)
        flag = False
        # Since the inner iteration could early terminate, thus `800` is unreachable
        for _ in range(800):
            tmpb = sub_matrix @ theta
            # get the norm of tmpb
            normb = np.linalg.norm(tmpb)
            # get the index of tmpb where are not zero and iterate on it
            nonzero = np.where(np.abs(theta) > 0.1)[0]
            thetab = None
            tmp_sub_matrix = sub_matrix.copy()
            # set columns in non_zero positions with 10 * np.ones
            tmp_sub_matrix[:, nonzero] = 10 * np.ones((sub_matrix.shape[0], len(nonzero)))
            for j1 in nonzero:
                tmpbb = tmpb - theta[j1] * sub_matrix[:, j1]
                # add
                tmpbbb = tmp_sub_matrix + tmpbb[:, np.newaxis]
                normbbb = np.linalg.norm(tmpbbb, axis=0)
                # get min value and index of normbbb
                min_index = np.argmin(normbbb)
                min_value = normbbb[min_index]
                if min_value < normb:
                    thetab = theta.copy()
                    thetab[j1] = 0
                    thetab[min_index] = 1
                    normb = min_value
                # sub
                tmpbbb =  tmpbb[:, np.newaxis] - tmp_sub_matrix
                normbbb = np.linalg.norm(tmpbbb, axis=0)
                # get min value and index of normbbb
                min_index = np.argmin(normbbb)
                min_value = normbbb[min_index]
                if min_value < normb:
                    thetab = theta.copy()
                    thetab[j1] = 0
                    thetab[min_index] = -1
                    normb = min_value
            # pbar update
            pbar.set_postfix({"norm": normb})
            if thetab is not None:
                theta = thetab
                continue
            else:
                flag = True
                break
        if flag:
            times += 1
        pbar.set_postfix({"Reach ending times": times})
    end = time.time()
    slap_time = end - start
    return slap_time

## Parameters

### Deep Recognition Model

In [4]:
# model path, where your model is
model_path = "models/ms1mv3_arcface_r100_fp16"
model_file = "backbone.pth"

In [5]:
# model parameters, need to revise if using different model
# for example, ms1mv3_arcface_r100_fp16 -> "r100", "fp16": True
kwargs = {"name": "r100", "dropout": 0.0, "fp16": True, "num_features": 512}
platform = "pytorch"  # pytorch ("onnx" is not supported currently)

In [6]:
backbone_model = BackBone(
    platform=platform, backbone_path=f"{model_path}/{model_file}", **kwargs
)

### Dataset

In [7]:
# fatial data path
data_fei_retinaface_path = "fei_face_dataset"
# data_color_feret_path = "/mnt/e/Downloads/colorferet/images_retina"
# data_lfw_retinaface_path = "/mnt/e/Downloads/lfw"
data_fatial_path = data_fei_retinaface_path  # which dataset do you use
data_set_estimated_noise_angle = 25  # hyper paramter, need to adjust according to the dataset(should slightly larger than the Noise in the dataset)
selected_subset_index = [1, 2, 3]  # fei dataset: [1, 2, 3] => 04, 05, 06
# fei dataset: [0, 2, 5] => 03, 05, 08

In [8]:
data_set_estimated_error_rate = utils.translate_error_angle_to_error_noise(
    data_set_estimated_noise_angle
)

### IronMask and Attack's Parameters

In [11]:
# parameter of codewords
dimension = 512
alpha = 16
k = 300  # the number of sampled linear equations (k, need to justify in paper section 5 according to noise)
threshold = 40  # threshold hyper-parameter(\theta_t, need to justify in paper section 5 according to noise, but generally 40 is enough)
use_precompute = True  # since the testing is time-consuming, we could just load the last precomputed results

## Load dataset

In [12]:
import tempfile

# tempdir: where to temporarily store fatial embeddings
tempdir = "tmp/" + model_path.split("/")[-1].split(".")[0]
tempdir

'tmp/ms1mv3_arcface_r100_fp16'

In [13]:
from torchvision import transforms
from dataset.fatial_dataset import FatialDataset

def transform_x(x):
    return x * 2 - 1

def transform_y(y):
    return str(y)

fatial_dataset = FatialDataset(data_fatial_path, transform=transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((112, 112)),
    transforms.ToTensor(),
    transform_x]), target_transform=transform_y)

In [14]:
from dataset.fatial_dataset_embedding_dict import FatialDataEmbeddingsDict
dataset_dict = FatialDataEmbeddingsDict(file_folder=tempdir, data_set=fatial_dataset, file_raw_name="labels.pickle",)
dataset_dict.dump_embeddings(backbone_model, 10, num_workers=0) # because of jupyter notebook's multi-threading issue, we set num_workers=0

In [15]:
identity_list = list(dataset_dict.keys())
print("Number of Identities:", len(identity_list))

Number of Identities: 200


## Solving original template against IronMask original scheme if sampled matrix is correct

ensure the matrix generated by linear equation sampler is "correct" as in Definition 4.1 in paper.

test success rate

In [16]:
import ironmask

runtimes_vec = []
success_times = 0
whole_iteration_times = 80000
index_list = selected_subset_index  # the index of the images to be used for puzzle solving, need to revise for different subsets
error_rate = data_set_estimated_error_rate  # the error rate of the templates, need to revise and estimate if using different datasets and models
if not use_precompute:
    for each_identity in (pbar := tqdm(identity_list)):
        each_isometric_matrixes = []
        each_cs = []
        for index in index_list:
            tmpcs = ironmask.sample_codeword(dimension, alpha)
            each_isometric_matrixes.append(
                ironmask.generate_secure_sketch(
                    dataset_dict[each_identity][index], tmpcs
                )
            )
            each_cs.append(tmpcs)
        assume_vector, b, run_times = attacker.solve_puzzle_with_n_matrix_known_places(
            each_isometric_matrixes,
            each_cs,
            dimension,
            alpha,
            threshold=threshold,
            max_rtimes=whole_iteration_times // len(identity_list),
            algorithm="LSA",
            disable_tqdm=False,
            k_each_matrix=k // 2,
            error_rate=error_rate * 3.0,
            return_runtimes=True,
        )
        if b is not None and (
            np.allclose(b, each_cs[0]) or np.allclose(b, -each_cs[0])
        ):
            success_times += 1
        pbar.set_postfix({"Success Rate": success_times / (pbar.n + 1)})
        runtimes_vec.append(run_times)

In [17]:
import pickle
if not use_precompute:
    with open("real-world/runtimes_vec_feret.pickle", "wb") as f:
        pickle.dump(runtimes_vec, f)
    with open("real-world/success_times_feret.pickle", "wb") as f:
        pickle.dump(success_times, f)

In [18]:
import pickle
from utils import subset_n_a_alpha

with open("real-world/runtimes_vec_feret.pickle", "rb") as f:
    runtimes_vec = pickle.load(f)
with open("real-world/success_times_feret.pickle", "rb") as f:
    success_times = pickle.load(f)

print("Success Rate: {:.2f}%".format(success_times / len(identity_list) * 100))
print(
    "Average Number of Run Tries(1/(pk * pf)): {:.2f}".format(
        np.sum(runtimes_vec) / success_times
    )
)

pkf = success_times / np.sum(runtimes_vec)
print("P_k * P_f  = {}".format(pkf))

Success Rate: 38.50%
Average Number of Run Tries(1/(pk * pf)): 815.60
P_k * P_f  = 0.0012260951258737918


Test lsa solver time

In [ ]:
from tqdm import tqdm

whole_time = 0
tmp_inner = 1
tmp_outer = len(identity_list)
for each_identity in (pbar := tqdm(identity_list, desc="Outer loop", disable=False)):
    each_isometric_matrixes = []
    each_cs = []
    for index in index_list:
        tmpcs = ironmask.sample_codeword(dimension, alpha)
        each_isometric_matrixes.append(
            ironmask.generate_secure_sketch(dataset_dict[each_identity][index], tmpcs)
        )
        each_cs.append(tmpcs)
    new_isometric_matrixes = [
        each_isometric_matrixes[i] @ each_isometric_matrixes[0].T
        for i in range(1, len(each_isometric_matrixes))
    ]
    new_cs = [each_cs[i] for i in range(1, len(each_cs))]
    puzzle = generate_puzzle_local_search_random(
        new_cs, new_isometric_matrixes, k=k // 2
    )
    whole_time += local_search_each_runtime(
        dimension, alpha, puzzle, disable_flag=True, each_runtime=tmp_inner
    )
    pbar.set_postfix({"Whole Time": whole_time})
tk = whole_time / (tmp_outer * tmp_inner)

In [24]:
print("Time(t_k) = {} miliseconds".format(tk * 1000))

Time(t_k) = 271.6527259349823 miliseconds


### Results

In [25]:
rk = utils.subset_n_a_alpha(dimension, k//2, alpha) * 2
print(rk)
et = tk / pkf * (2**rk)
print("Whole Time:")
print("* seconds: {:.2f}".format(et))
print("* minutes: {:.2f}".format(et / 60))
print("* hours: {:.2f}".format(et / 3600))
print("* days: {:.2f}".format(et / 3600 / 24))
print("* years: {:.2f}".format(et / 3600 / 24 / 365))

16.292172969044763
Whole Time:
* seconds: 17779627.57
* minutes: 296327.13
* hours: 4938.79
* days: 205.78
* years: 0.56
